In [52]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u

In [53]:
def cal_snr(
        N_star: float,  
        N_sky: float, 
        N_dark: float, 
        N_read: float, 
):
    """
    Calculate the SNR of a spectrum.
    
    Parameters
    ----------
    N_star : float
        Number of photons from the star.
    N_sky : float
        Number of photons from the sky.
    N_dark : float
        Number of photons from the dark current.
    N_read : float
        Number of photons from the read noise.
        
    Returns
    -------
    snr : float
        SNR of the spectrum.
    """
    snr = N_star / np.sqrt(N_star + (N_sky + N_dark + N_read**2))
    return snr

def cal_flux(mag, k, eff):
    return 10**(-0.4 * (mag - 8.9 + k)) * eff * u.Jy

In [54]:
pix_scale = 0.6 * u.arcsec  # arcsec/pixel
pix_size = 10e-6 * u.meter  # m/pixel

In [55]:
seeing = 1.5 * u.arcsec

r_aperture = 1.5 * (seeing / 2)
r_star = seeing / 2

field_aperture = np.pi * r_aperture**2
pix_aperture = field_aperture / pix_scale**2
area_aperture = pix_aperture * pix_size**2

field_star = np.pi * r_star**2
# field_star = field_aperture
pix_star = field_star / pix_scale**2
area_star = pix_star * pix_size**2

In [66]:
mag_star = 20
mag_sky = 21
kV = 0.35
airmass = 1.2
tele_eff = 0.4

In [67]:
F_star = cal_flux(mag_star, kV * airmass, tele_eff)
F_sky = cal_flux(mag_sky, kV * airmass, tele_eff)

F_star, F_sky

(<Quantity 2.13825744e-05 Jy>, <Quantity 8.51255618e-06 Jy>)

quantum eff

In [68]:
wavelength_arr = np.array([350, 400, 500, 650, 900]) * u.nm
qe_arr = np.array([0.3, 0.75, 0.75, 0.8, 0.5]) * u.electron / u.photon
# 线性插值量子效率
func_qe = lambda l: np.interp(l, wavelength_arr, qe_arr)

In [69]:
wavelength = 550 * u.nm
band_width = 100 * u.nm
# band_width_nu = np.abs(3e8 / (wavelength - band_width / 2) - 3e8 / (wavelength + band_width / 2))
freq_width = (3e8 * u.m / u.second / wavelength**2 * band_width).to(u.Hz)

qe = func_qe(wavelength)
qe

<Quantity 0.76666667 electron / ph>

In [70]:
# plank constant
h = 6.626e-34 * u.J * u.second
# energy per photon
E = h * 3e8 * u.m /u.second / wavelength / u.photon

t_exp = 20 * u.second

area_tele = np.pi * (1 * u.meter)**2

N_star = F_star * area_tele * freq_width / E * qe * t_exp
N_sky = F_sky * area_tele * freq_width / E * qe * t_exp

# simplify unit
N_star.to(u.electron), N_sky.to(u.electron)

(<Quantity 2826.3915104 electron>, <Quantity 1125.20672708 electron>)

In [71]:
eff_dark = 4 * u.electron / u.hour    # e-/pix/hour
N_eff = eff_dark * t_exp * pix_aperture
N_eff.to(u.electron)

<Quantity 0.24543693 electron>

In [72]:
eff_read = 4 * u.electron
N_read = eff_read * np.sqrt(pix_aperture)
N_read

<Quantity 13.29340388 electron>

In [73]:
snr = cal_snr(N_star.to(u.electron).value, N_sky.to(u.electron).value, N_eff.to(u.electron).value, N_read.to(u.electron).value)
snr

43.98788805333756

In [77]:
180 * 3600 * 10e-6 * u.meter / np.pi / (0.6)

<Quantity 3.43774677 m>